In [1]:
import numpy as np
from torch.utils.data import DataLoader
from functions import *
from torch.nn import GRU,Conv1d,Linear,ReLU,Sequential,MSELoss,L1Loss,LeakyReLU,Dropout2d,Dropout
from torch.optim import Adam
from pytorch_util import *
from pytorch_models import ConvBatchRelu1D,GRU_NCL

In [2]:
long_page = np.load('../Data/long_page.npy')
long_value = np.load('../Data/long_value.npy')
start_index = np.load('../Data/start_index.npy')
stds = np.load('../Data/stds_log.npy')
means = np.load('../Data/means_log.npy')
long_normal_value = np.load('../Data/long_lognormal_value.npy')

In [3]:
# to account for look back period 365 + 60
start_index = start_index + 430

In [4]:
## to change ##
batch_size = 64
clip = .5

In [5]:
# baseline model
# yhat = long_normal_value[:,803-61:803-60]
# y = long_normal_value[:,803-60:]
# SMAPE_np(y,yhat)
# >> 106.31
# yhat = long_normal_value[:,803-61:803-60]
# y = long_normal_value[:,803-60:]
# np.mean(np.square(y - yhat))
# >>> 1.1114173

In [6]:
# set up data loader
train_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means,useMean=True)
train_gen = DataLoader(train_gen,batch_size,True,num_workers=3,drop_last=True)

val_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means,random=False,useMean=True)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=3)

train_gen_nor = SequenceGenNormalized(long_normal_value,start_index)
train_gen_nor = DataLoader(train_gen_nor,batch_size,True,num_workers=3,drop_last=True)

val_gen_nor = SequenceGenNormalized(long_normal_value,start_index,random=False)
val_gen_nor = DataLoader(val_gen_nor,batch_size)

In [7]:
conv = Sequential(ConvBatchRelu1D(16,32,3),
                  Dropout2d(0.5),
                  ConvBatchRelu1D(32,64,3),
                  Dropout2d(0.5),
                  ConvBatchRelu1D(64,128,3),
                  Dropout2d(0.5))

In [8]:
rnn = GRU_NCL(3,8,2,batch_first=True,dropout=0.5,bidirectional=True)

In [9]:
seq_model = Sequential(rnn,conv)

In [10]:
linear = Sequential(Linear(128,64),LeakyReLU(0.1,True),Dropout(0.5),Linear(64,1))

In [11]:
convert = Sequential(Linear(114,60),LeakyReLU(0.1,True),Dropout(0.5))

In [12]:
# model trained on normalized data
model_nor = CNN_RNN2seq_normalized_NCL(seq_model,linear,convert).to('cuda:0')

In [13]:
loss_func_nor = loss_func_generator_normalized(MSELoss())
# loss_func_nor = loss_func_generator_normalized(L1Loss())
#loss_func_nor = loss_func_generator_normalized(SMAPE)

In [14]:
opt = Adam(trainable_parameter(model_nor),lr=1e-3)

In [15]:
model_nor = fit(3, model_nor, loss_func_nor, opt, train_gen_nor, val_gen_nor,clip,patience=0)

epoch:0, train_loss:0.7780498194725403, val_loss:0.8926210999488831
epoch:1, train_loss:0.7368772181822228, val_loss:0.8772290945053101
epoch:2, train_loss:0.7239208766061407, val_loss:0.864576518535614
Training completed in 133.71085906028748s


In [16]:
model_nor = fit(3, model_nor, loss_func_nor, opt, train_gen_nor, val_gen_nor,clip,lossBest=0.864576518535614)

epoch:0, train_loss:0.7201865558488382, val_loss:0.8715601563453674
epoch:1, train_loss:0.7132975118611143, val_loss:0.8656455278396606
epoch:2, train_loss:0.7031019821222582, val_loss:0.8500885963439941
Training completed in 134.23873353004456s


In [18]:
model = CNN_RNN2seq_logNormalized_NCL(seq_model,linear,convert).to('cuda:0')

In [19]:
opt = Adam(trainable_parameter(model),lr=1e-4)

In [20]:
loss_func = loss_func_generator(SMAPE)

In [21]:
model = fit(5, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:35.95565543792409, val_loss:38.30767822265625
epoch:1, train_loss:35.83657377075037, val_loss:38.16237258911133
epoch:2, train_loss:35.80459126487297, val_loss:38.00175857543945
epoch:3, train_loss:35.75376296166929, val_loss:37.59684753417969
epoch:4, train_loss:35.73306873104115, val_loss:37.68037414550781
Training completed in 242.75394749641418s
